<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/nl_small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-dj8b7yez
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-dj8b7yez
  Resolved https://github.com/huggingface/transformers to commit b074461ef0f54ce37c5239d30ee960ece28d11ec
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.0 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8001040 sha256=e6ce6d720e01b56544a0868171b94b8abb29b7742ff5de6c6e9ebc2b64e4e448
  Stored in directory: /tmp/pip-ephem-wheel-cache-zglqgkfg/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfull

In [2]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [3]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [4]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "nl", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("small")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 19.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6

In [6]:
model = whisper.load_model("small")

100%|███████████████████████████████████████| 461M/461M [00:08<00:00, 56.7MiB/s]


In [7]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [8]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="nl")
    return result["text"]

In [9]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [10]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score


In [11]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
count = 0

for sample in voxpopuli_data:
    audio_file_path = sample['audio']['path']
    ground_truth_text = sample['normalized_text']

    # Transcribe the audio
    transcription = transcribe_audio(audio_file_path)

    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    count += 1

    print(f"Processed {count} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")
    # Calculate average metrics
average_wer = total_wer / count
average_precision = total_precision / count
average_recall = total_recall / count
average_f1 = total_f1 / count

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")


Processed 1 files, WER: 0.16666666666666666, Precision: 0.84, Recall: 0.84, F1: 0.8399999999999999
Processed 2 files, WER: 0.3076923076923077, Precision: 0.75, Recall: 0.84, F1: 0.7924528301886793
Processed 3 files, WER: 0.25, Precision: 0.7352941176470589, Recall: 0.7352941176470589, F1: 0.735294117647059
Processed 4 files, WER: 0.3, Precision: 0.75, Recall: 0.75, F1: 0.75
Processed 5 files, WER: 0.47368421052631576, Precision: 0.5333333333333333, Recall: 0.47058823529411764, F1: 0.5
Processed 6 files, WER: 0.21428571428571427, Precision: 0.8571428571428571, Recall: 0.9230769230769231, F1: 0.888888888888889
Processed 7 files, WER: 0.2631578947368421, Precision: 0.7272727272727273, Recall: 0.7058823529411765, F1: 0.7164179104477613
Processed 8 files, WER: 0.2222222222222222, Precision: 0.8125, Recall: 0.7647058823529411, F1: 0.787878787878788
Processed 9 files, WER: 0.6, Precision: 0.47368421052631576, Recall: 0.5, F1: 0.4864864864864865
Processed 10 files, WER: 0.24390243902439024, Pr